In [7]:
import os
import json

In [8]:
from datasets import load_dataset 

dataset = load_dataset("multi_news",split="train")

os.mkdir("./Dataset")

Found cached dataset multi_news (C:/Users/Francisco/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72)


In [28]:
dataset.to_json("C:/Users/Francisco/Desktop/SmartNews/nlp_section/Dataset/train.json")

Creating json from Arrow format: 100%|██████████| 45/45 [00:04<00:00,  9.63ba/s]


571553657

<h1 style="color:red"> <b> Usar para retirar unicode </h1>

In [33]:
data = []
with open("./Dataset/train.json") as file:
    for line in dataset:
            jsonObj = json.loads(line)
            document = jsonObj["document"]
            summary = jsonObj["summary"]
            data.append((document, summary))

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [34]:
from tokenizers import BertWordPieceTokenizer

wordPiece = BertWordPieceTokenizer()

wordPiece.train("./Dataset/train.json")

In [35]:
#os.mkdir('./tokenizer')

wordPiece.save_model('./tokenizer')

['./tokenizer\\vocab.txt']

In [3]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('./tokenizer')

In [9]:
import torch

def mlm(tensor):
    rand = torch.rand(tensor.shape) #todos os valores estao entre 0 e 1
    mask_arr =(rand < 0.15) * (tensor > 2 ) #mask tokens com crit abaixo de 15% e q n usa os special tokens
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i, selection] = 5
    return tensor

<h1>Apagar</h1>

In [48]:
with open('./Dataset/train.json','r',encoding='utf-8') as fp:
    lines = fp.read().split('\n')

from tqdm.auto import tqdm

for line in tqdm(lines):
 sample = tokenizer(line)

labels = torch.tensor([x for x in sample['input_ids']])
mask = torch.tensor([x for x in sample['attention_mask']])


  0%|          | 39/44973 [00:01<22:41, 33.01it/s]


KeyboardInterrupt: 

In [26]:
input_ids = labels.detach().clone()
rand = torch.rand(input_ids.shape)
mask_arr = (rand < .15) * (input_ids > 2)
for i in range(input_ids.shape[0]):
    selection = torch.flatten(mask_arr[i].nonzero()).tolist()
    input_ids[i,selection] = 5 #[MASK] token

AttributeError: 'list' object has no attribute 'detach'